In [8]:
from torf import Torrent
import hashlib
from math import ceil
import bencodepy

In [3]:
import hashlib


def _read_piece(torrent, piece_idx):
    if torrent.mode == "singlefile":
        offset = piece_idx * torrent.piece_size
        try:
            with open(torrent.name, "rb") as file:
                file.seek(offset)
                data = file.read(torrent.piece_size)
                if not data:
                    return None
                return data
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            return None
        except IOError as e:
            print(f"Error reading file: {e}")
            return None

    else:
        # Handle multi-file mode (if needed)
        pass


torrent = Torrent.read("compnet.torrent")

print(torrent.name)

data = _read_piece(torrent, 2)
print(len(data))
hash = hashlib.sha1(data).digest()
if hash == torrent.hashes[2]:
    print("Piece hash matches")

compnet.pdf
524288
Piece hash matches


In [4]:
import os

torrent = Torrent.read("asm1.torrent")

info_hash = torrent.infohash


def _check_local_repo(info_hash):

    torrent_files = [f for f in os.listdir("./") if f.endswith(".torrent")]

    print(torrent_files)

    for torrent_file in torrent_files:

        torrent = Torrent.read(torrent_file)

        if torrent.infohash == info_hash:

            return torrent_file

    return None


_check_local_repo(info_hash=info_hash)

['asm1.torrent', 'compnet.torrent']


'asm1.torrent'

In [79]:
client_prefix = "-ST0001-"
import random

# random number 12 digits
unique_component = str(random.randint(0, 999999999999)).zfill(12)
id = f"{client_prefix}{unique_component}"

In [107]:
import struct

print(len(id))
info_hash = torrent.infohash
pstrlen = struct.pack("B", 19)
pstr = b"BitTorrent protocol"
reserved = b"\x00" * 8
info_hash = bytes.fromhex(info_hash)
print(len(info_hash))
handshake = pstrlen + pstr + reserved + info_hash + id.encode("utf-8")

print(handshake)
print(len(handshake))

20
20
b'\x13BitTorrent protocol00000000G\xee\xd7?ZK\r\x0c!\xa3\xc3\x81\x00\xcbA\xb4q@:\x80-ST0001-234897642375'
68


In [112]:
pstrlen = struct.unpack("B", handshake[0:1])[0]
pstr = handshake[1:20]
reserved = handshake[20:28]
info_hash = handshake[28:48]
peer_id = handshake[48:]

print(pstrlen)
print(pstr.decode())
print(reserved.hex())
print(info_hash.hex())
print(peer_id.decode("utf-8"))

print(torrent.infohash)

19
BitTorrent protocol
3030303030303030
47eed73f5a4b0d0c21a3c38100cb41b471403a80
-ST0001-234897642375
47eed73f5a4b0d0c21a3c38100cb41b471403a80


In [3]:
tor = Torrent.read("TO_BE_SHARED.torrent")

MetainfoError: Invalid metainfo: Missing 'piece length' in ['info']

In [11]:
data = None
with open("TO_BE_SHARED.torrent", "rb") as f:
    data = bencodepy.decode(f.read())

print(data.keys())

odict_keys([b'announce', b'info'])


In [18]:
data[b"info"]

OrderedDict([(b'piece_length', 524288),
             (b'pieces',
              b'\xd3\xe6\x06GQ\xb7\xb1okT\xc0\x1en;A\xbd\xbe\xb8c\xad(^\x0cX\xa6\x13C\xe0|Wv\xb0\xc7\xaeY\xaf\x98\x1c\x1a\xee\x01Z\xeb\xd3?\x9a-k\x8bRj$\x06d\xe1\x1c\xfe\xa1\x08\x1b\x8f\xdeEz\xa3x\xd7\x03\xcd\x07u\x88\x8d^g\xa7L\xfa\xfb\xa0S\xa2\x84\x98\x81z\x810nr\xf8\xc7\xef\xfe\x95\xaeX\xf7!\xb7\xfc\x81\xabO&\xd4\xcdL\'\x9d\xfa\x15)\xe1\xa16#\xc5\x03\x1d\xc9?r~i\xe8Qa?\x15\xadcK\xd7\xf3\xb1\x08@\xf4S\xc5\x8f\xf5y\x986G\'kc\x0f\x8f\xa3\x89\xa0\x12\x90\xf4\xee\nR\xa0\x0b\x84aY<\xe34\t\xd7\xc5\xd0A\x16\x99\xcb\xf9\xcd\xa3\xa5K\x86"\xc3\x18=\x95dB#\xac\xf4\x0b\x97\xb4\'\x1a\xb3\x9e\xda9\xa3\xee^kK\r2U\xbf\xef\x95`\x18\x90\xaf\xd8\x07\t'),
             (b'name', b'TO_BE_SHARED'),
             (b'files',
              [OrderedDict([(b'length', 4176243),
                            (b'path',
                             [b'TO_BE_SHARED', b'shared', b'Task 4.pdf'])]),
               OrderedDict([(b'length', 27),
              

In [39]:
hash1 = data[b"info"][b"pieces"][20:40]
hash1

b'(^\x0cX\xa6\x13C\xe0|Wv\xb0\xc7\xaeY\xaf\x98\x1c\x1a\xee'

In [26]:
tor = Torrent.read("MY_TO_BE_SHARED.torrent")

In [40]:
tor.hashes[2]

b'\x01Z\xeb\xd3?\x9a-k\x8bRj$\x06d\xe1\x1c\xfe\xa1\x08\x1b'

In [48]:
print(data[b"info"][b"pieces"][100:120] == tor.hashes[5])

True
